Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Extractive Summarization on CNN/DM Dataset using Transformer Version of BertSum


### Summary

This notebook demonstrates how to fine tune Transformers for extractive text summarization. Utility functions and classes in the NLP Best Practices repo are used to facilitate data preprocessing, model training, model scoring, result postprocessing, and model evaluation.

BertSum refers to  [Fine-tune BERT for Extractive Summarization](https://arxiv.org/pdf/1903.10318.pdf) with [published example](https://github.com/nlpyang/BertSum/). And the Transformer version of Bertsum refers to our modification of BertSum and the source code can be accessed at (https://github.com/daden-ms/BertSum/). 

Extractive summarization are usually used in document summarization where each input document consists of mutiple sentences. The preprocessing of the input training data involves assigning label 0 or 1 to the document sentences based on the give summary. The summarization problem is also simplfied to classifying whether a document sentence should be included in the summary. 

The figure below illustrates how BERTSum can be fine tuned for extractive summarization task. [CLS] token is inserted at the beginning of each sentence, so is [SEP] token at the end. Interval segment embedding and positional embedding are added upon the token embedding as the input of the BERT model. The [CLS] token representation is used as sentence embedding and only the [CLS] tokens are used as the input for the summarization model. The summarization layer predicts the probability for each  sentence being included in the summary. Techniques like trigram blocking can be used to improve model accuarcy.   

<img src="https://nlpbp.blob.core.windows.net/images/BertSum.PNG">


### Before You Start

The running time shown in this notebook is on a Standard_NC24s_v3 Azure Ubuntu Virtual Machine with 4 NVIDIA Tesla V100 GPUs. 
> **Tip**: If you want to run through the notebook quickly, you can set the **`QUICK_RUN`** flag in the cell below to **`True`** to run the notebook on a small subset of the data and a smaller number of epochs. 

Using only 1 NVIDIA Tesla V100 GPUs, 16GB GPU memory configuration,
- for data preprocessing, it takes around 1 minutes to preprocess the data for quick run. Otherwise it takes ~20 minutes to finish the data preprocessing. This time estimation assumes that the chosen transformer model is "distilbert-base-uncased" and the sentence selection method is "greedy", which is the default. The preprocessing time can be significantly longer if the sentence selection method is "combination", which can achieve better model performance.

- for model fine tuning, it takes around 2 minutes for quick run. Otherwise, it takes around ~3 hours to finish. This estimation assumes the chosen encoder method is "transformer". The model fine tuning time can be shorter if other encoder method is chosen, which may result in worse model performance. 

### Additional Notes

* **ROUGE Evalation**: To run rouge evaluation, please refer to the section of compute_rouge_perl in [summarization_evaluation.ipynb](./summarization_evaluation.ipynb) for setup.

* **Distributed Training**:
Please note that the jupyter notebook only allows to use pytorch [DataParallel](https://pytorch.org/docs/master/nn.html#dataparallel). Faster speed and larger batch size can be achieved with pytorch [DistributedDataParallel](https://pytorch.org/docs/master/notes/ddp.html)(DDP). Script [extractive_summarization_cnndm_distributed_train.py](./extractive_summarization_cnndm_distributed_train.py) shows an example of how to use DDP.



In [2]:
%load_ext autoreload

%autoreload 2

In [3]:
## Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of epochs.
QUICK_RUN = False
## Set USE_PREPROCSSED_DATA = True to skip the data preprocessing
USE_PREPROCSSED_DATA = True

### Configuration


In [4]:
import os
import shutil
import sys
from tempfile import TemporaryDirectory
import torch

nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.dataset.cnndm import CNNDMBertSumProcessedData, CNNDMSummarizationDataset
# from utils_nlp.dataset.bundesministerium import BUNDBertSumProcessedData, BUNDSummarizationDataset
from utils_nlp.dataset.swiss import SwissSummarizationDataset

from utils_nlp.eval import compute_rouge_python, compute_rouge_perl
from utils_nlp.models.transformers.extractive_summarization import (
    ExtractiveSummarizer,
    ExtSumProcessedData,
    ExtSumProcessor,
)

from utils_nlp.models.transformers.datasets import SummarizationDataset
import nltk
from nltk import tokenize

import pandas as pd
import scrapbook as sb
import pprint

/home/ubuntu/anaconda3/envs/nlp_gpu/lib/python3.6/site-packages/dask/dataframe/utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm



### Configuration: choose the transformer model to be used

Several pretrained models have been made available by [Hugging Face](https://github.com/huggingface/transformers). For extractive summarization, the following pretrained models are supported. 

In [5]:
pd.DataFrame({"model_name": ExtractiveSummarizer.list_supported_models()})

,model_name
0,bert-base-uncased
1,bert-base-german-cased
2,distilbert-base-uncased
3,dbmdz/bert-base-german-uncased
4,bert-base-german-dbmdz-cased
5,bert-base-multilingual-cased
6,distilbert-base-german-cased
7,bert-base-german-dbmdz-uncased
8,severinsimmler/bert-adapted-german-press
9,xlm-roberta-large-finetuned-conll03-german


In [6]:
# Transformer model being used
MODEL_NAME = "severinsimmler/bert-adapted-german-press"

# Data being used
DATA_NAME = "swiss"

In [16]:
# notebook parameters
# the cache data path during find tuning
CACHE_DIR = TemporaryDirectory().name
processor = ExtSumProcessor(model_name=MODEL_NAME, cache_dir=CACHE_DIR)

### Data Preprocessing

The dataset we used for this notebook is CNN/DM dataset which contains the documents and accompanying questions from the news articles of CNN and Daily mail. The highlights in each article are used as summary. The dataset consits of ~289K training examples, ~11K valiation examples and ~11K test examples.  You can choose the [Option 1] below preprocess the data or [Option 2] to use the preprocessed version at [BERTSum published example](https://github.com/nlpyang/BertSum/). You don't need to manually download any of these two data sets as the code below will handle downloading. Functions defined specific in [cnndm.py](../../utils_nlp/dataset/cnndm.py) are unique to CNN/DM dataset that's preprocessed by harvardnlp. However, it provides a skeleton of how to preprocessing text into the format that model preprocessor takes: sentence tokenization and work tokenization. 

##### Details of Data Preprocessing

The purpose of preprocessing is to process the input articles to the format that model finetuning needed. Assuming you have (1) all articles and (2) target summaries, each in a file and line-breaker separated, the steps to preprocess the data are:
1. sentence tokenization
2. word tokenization
3. **label** the sentences in the article with 1 meaning the sentence is selected and 0 meaning the sentence is not selected. The algorithms for the sentence selection are "greedy" and "combination" and can be found in [sentence_selection.py](../../utils_nlp/dataset/sentence_selection.py)
3. convert each example to  the desired format for extractive summarization
    - filter the sentences in the example based on the min_src_ntokens argument. If the lefted total sentence number is less than min_nsents, the example is discarded.
    - truncate the sentences in the example if the length is greater than max_src_ntokens
    - truncate the sentences in the example and the labels if the total number of sentences is greater than max_nsents
    - [CLS] and [SEP] are inserted before and after each sentence
    - wordPiece tokenization or Byte Pair Encoding (BPE) subword tokenization
    - truncate the example to 512 tokens
    - convert the tokens into token indices corresponding to the transformer tokenizer's vocabulary.
    - segment ids are generated and added
    - [CLS] token positions are logged
    - [CLS] token labels are truncated if it's greater than 512, which is the maximum input length that can be taken by the transformer model.
    
    
Note that the original BERTSum paper use Stanford CoreNLP for data preprocessing, here we use NLTK for data preprocessing. 

In [17]:
# the data path used to save the downloaded data file
DATA_PATH = '/home/ubuntu/mnt/data/swiss_processed'
# The number of lines at the head of data file used for preprocessing. -1 means all the lines.
TOP_N = 1000
if not QUICK_RUN:
    TOP_N = -1

##### [Option 1] Preprocess  data (Please skip this part if you choose to use preprocessed data)
The code in following cell will download the CNN/DM dataset listed at https://github.com/harvardnlp/sent-summary/.

In [18]:
if DATA_NAME is "cnndm":
    train_dataset, test_dataset = CNNDMSummarizationDataset(top_n=TOP_N, local_cache_path=DATA_PATH)
elif DATA_NAME is "swiss":
    train_dataset, validation_dataset, test_dataset = SwissSummarizationDataset(top_n=TOP_N, validation=True, language='german')


In [19]:
len(train_dataset), len(validation_dataset), len(test_dataset)

(85500, 9500, 5000)

### Preprocess the data.

In [20]:

ext_sum_train = processor.preprocess(train_dataset, oracle_mode="greedy")
ext_sum_test = processor.preprocess(test_dataset, oracle_mode="greedy")


### Save the data.

In [21]:
SAVE_DATA = True


# save and load preprocessed data

if SAVE_DATA:
    save_path = os.path.join(DATA_PATH, DATA_NAME + "_processed")
    os.makedirs(save_path, exist_ok=True)

    torch.save(ext_sum_train, os.path.join(save_path, "train_full.pt"))
    torch.save(ext_sum_test, os.path.join(save_path, "test_full.pt"))

In [ ]:
len(ext_sum_train)

In [ ]:
len(ext_sum_test)

#### Inspect Data

In [ ]:
ext_sum_train[0]

In [ ]:
ext_sum_train[0].keys()

##### [Option 2] Reuse cached preprocessed data

In [ ]:
if USE_PREPROCSSED_DATA:
    save_path = os.path.join(DATA_PATH, DATA_NAME + "_processed")
    ext_sum_train = torch.load(os.path.join(save_path, "train_full.pt"))
    ext_sum_test = torch.load(os.path.join(save_path, "test_full.pt"))
    

In [9]:
ext_sum_train[0]

{'src': [['Im',
   'März',
   '2012',
   'wurde',
   'er',
   'im',
   'finnischen',
   'Kuopio',
   'erstmals',
   'im',
   'zweitklassigen',
   'Continental',
   'Cup',
   'eingesetzt',
   '.'],
  ['Beim', 'Sommerspringen', 'am', '16', '.'],
  ['September',
   '2012',
   'auf',
   'der',
   'neu',
   'erbauten',
   'Sneschinka',
   'im',
   'russischen',
   'Tschaikowski',
   'sprang',
   'er',
   'als',
   'Dritter',
   'erstmals',
   'in',
   'diesem',
   'Wettbewerb',
   'auf',
   'das',
   'Podium',
   '.'],
  ['Eine',
   'Woche',
   'später',
   'konnte',
   'er',
   'im',
   'sächsischen',
   'Klingenthal',
   'sogar',
   'Platz',
   'zwei',
   'belegen',
   '.'],
  ['Am',
   'Ende',
   'reichte',
   'es',
   'in',
   'der',
   'Gesamtwertung',
   'des',
   'Sommer-Grand-Prix',
   'zum',
   'fünften',
   'Platz',
   '.'],
  ['Aufgrund',
   'dieser',
   'Erfolge',
   'wurde',
   'er',
   'für',
   'den',
   'Weltcupauftakt',
   'am',
   'letzten',
   'Novemberwochenende',
 

### Model training
To start model training, we need to create a instance of ExtractiveSummarizer.

Potentionally, roberta-based model and xlnet can be supported but needs to be tested.
#### Choose the encoder algorithm.
There are four options:
- baseline: it used a smaller transformer model to replace the bert model and with transformer summarization layer
- classifier: it uses pretrained BERT and fine-tune BERT with **simple logistic classification** summarization layer
- transformer: it uses pretrained BERT and fine-tune BERT with **transformer** summarization layer
- RNN: it uses pretrained BERT and fine-tune BERT with **LSTM** summarization layer

In [10]:

if USE_PREPROCSSED_DATA: #if bertsum published data is used
    BATCH_SIZE = 5 # 3000 # batch size, unit is the number of tokens
    MAX_POS_LENGTH = 512
else:
    BATCH_SIZE = 5 # batch size, unit is the number of samples
    MAX_POS_LENGTH = 512


# GPU used for training
NUM_GPUS = torch.cuda.device_count()

# Encoder name. Options are: 1. baseline, classifier, transformer, rnn.
ENCODER = "transformer"

# Learning rate
LEARNING_RATE=2e-3

# How often the statistics reports show up in training, unit is step.
REPORT_EVERY=50

# total number of steps for training
MAX_STEPS=1e2
# number of steps for warm up
WARMUP_STEPS=5e2
    
if not QUICK_RUN:
    MAX_STEPS=5e4
    WARMUP_STEPS=5e3
 

In [11]:
summarizer = ExtractiveSummarizer(processor, MODEL_NAME, ENCODER, MAX_POS_LENGTH, CACHE_DIR)

In [12]:
summarizer.model_name

'severinsimmler/bert-adapted-german-press'

In [13]:
#"""

summarizer.fit(
            ext_sum_train,
            num_gpus=NUM_GPUS,
            batch_size=BATCH_SIZE,
            gradient_accumulation_steps=2,
            max_steps=MAX_STEPS,
            learning_rate=LEARNING_RATE,
            warmup_steps=WARMUP_STEPS,
            verbose=True,
            report_every=REPORT_EVERY,
            clip_grad_norm=False,
            use_preprocessed_data=False
        )

#"""


Iteration:   1%|          | 100/17100 [00:26<1:18:10,  3.62it/s]

timestamp: 14/07/2020 10:39:06, average loss: 27.062008, time duration: 26.768522,
                            number of examples in current reporting: 500, step 50
                            out of total 100


Iteration:   1%|          | 200/17100 [00:53<1:18:07,  3.61it/s]

timestamp: 14/07/2020 10:39:33, average loss: 24.955555, time duration: 26.455107,
                            number of examples in current reporting: 500, step 100
                            out of total 100


Iteration:   1%|          | 201/17100 [00:53<1:15:59,  3.71it/s]


In [14]:
summarizer.save_model(
    os.path.join(
        CACHE_DIR,
        "extsum_modelname_{0}_usepreprocess{1}_steps_{2}.pt".format(
            MODEL_NAME, USE_PREPROCSSED_DATA, MAX_STEPS
        ),
    )
)

saving through pytorch
saving as pickle


FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmp104y8y_d/extsum_modelname_severinsimmler/bert-adapted-german-press_usepreprocessTrue_steps_100.0.pt'

In [15]:
# for loading a previous saved model
"""
import torch
model_path = os.path.join(
        CACHE_DIR,
        "extsum_modelname_{0}_usepreprocess{1}_steps_{2}.pt".format(
            MODEL_NAME, USE_PREPROCSSED_DATA, MAX_STEPS
        ))
summarizer = ExtractiveSummarizer(processor, MODEL_NAME, ENCODER, MAX_POS_LENGTH, CACHE_DIR)
summarizer.model.load_state_dict(torch.load(model_path, map_location="cpu"))
"""

'\nimport torch\nmodel_path = os.path.join(\n        CACHE_DIR,\n        "extsum_modelname_{0}_usepreprocess{1}_steps_{2}.pt".format(\n            MODEL_NAME, USE_PREPROCSSED_DATA, MAX_STEPS\n        ))\nsummarizer = ExtractiveSummarizer(processor, MODEL_NAME, ENCODER, MAX_POS_LENGTH, CACHE_DIR)\nsummarizer.model.load_state_dict(torch.load(model_path, map_location="cpu"))\n'

### Model Evaluation

[ROUGE](https://en.wikipedia.org/wiki/ROUGE_(metric)), or Recall-Oriented Understudy for Gisting Evaluation has been commonly used for evaluating text summarization.

In [16]:
ext_sum_test[0].keys()

dict_keys(['src', 'src_txt', 'tgt', 'tgt_txt', 'oracle_ids'])

In [17]:
if "segs" in ext_sum_test[0]: # preprocessed_data
    source = [i['src_txt'] for i in ext_sum_test]
    target = ["\n".join(i['tgt_txt'].split("<q>")) for i in ext_sum_test]
else:
    source = []
    temp_target = []
    for i in ext_sum_test:
        source.append(i["src_txt"]) 
        temp_target.append(" ".join(j) for j in i['tgt']) 
    target = [''.join(i) for i in list(temp_target)]

In [18]:
%%time
sentence_separator = "\n"
prediction = summarizer.predict(ext_sum_test, num_gpus=NUM_GPUS, batch_size=256, sentence_separator=sentence_separator)

Scoring: 100%|██████████| 20/20 [02:07<00:00,  5.69s/it]


CPU times: user 2min 5s, sys: 14.4 s, total: 2min 20s
Wall time: 2min 9s


In [19]:
len(prediction)

5000

In [ ]:
rouge_scores = compute_rouge_python(cand=prediction, ref=target)
pprint.pprint(rouge_scores)

In [ ]:
target[0]

In [ ]:
prediction[0]

In [ ]:
source[0]

In [ ]:
# for testing
sb.glue("rouge_2_f_score", rouge_scores['rouge-2']['f'])

## Prediction on a single input sample

In [ ]:
source = """
Am Morgen des 5. September überfielen acht bewaffnete arabische Terroristen, die im Vorfeld von deutschen Neonazis unterstützt worden waren, ein Wohnquartier des israelischen Teams im olympischen Dorf. Ein israelischer Sportler wurde schon während des Angriffs getötet, ein weiterer erlag kurz darauf seinen Verwundungen. Die übrigen neun Mannschaftsmitglieder wurden als Geiseln genommen. Die Geiselnehmer verlangten zunächst die Freilassung von 232 Palästinensern und des japanischen Terroristen Kōzō Okamoto aus israelischer sowie der RAF-Mitglieder Andreas Baader und Ulrike Meinhof aus deutscher Haft. Die israelische Regierung unter Golda Meir lehnte die an sie gestellte Forderung ab. Versuche deutscher Politiker, sich als Austauschgeiseln anzubieten, wiesen die Palästinenser zurück. In der Nacht vom 5. auf den 6. September unternahm die bayerische Polizei auf dem Militärflugplatz Fürstenfeldbruck einen schlecht geplanten und durchgeführten[1] Befreiungsversuch, der vollständig scheiterte.

Die drei überlebenden Terroristen wurden schon wenige Wochen nach ihrer Tat mit einer Flugzeugentführung freigepresst. Infolgedessen war das Olympia-Attentat nie Gegenstand eines ordentlichen Gerichtsverfahrens. Die israelische Regierung ordnete eine Vergeltungsaktion „Zorn Gottes“ durch die Sondereinheit „Caesarea“ des Mossad an. Diese tötete in den Jahren nach 1972 etwa zwanzig Palästinenser, die direkt oder indirekt an dem Anschlag beteiligt gewesen waren, aber auch Unschuldige.[2] Nach dem katastrophalen Ausgang des Geiseldramas stellte die deutsche Bundesregierung die Antiterror-Spezialeinheit Bundesgrenzschutzgruppe 9 (GSG 9) auf. Die Länderpolizeien folgten mit Spezialeinsatzkommandos (SEK)."""

In [ ]:
test_dataset = SummarizationDataset(
    None,
    source=[source],
    source_preprocessing=[tokenize.sent_tokenize],
    word_tokenize=nltk.word_tokenize,
    language='german'
)
processor = ExtSumProcessor(model_name=MODEL_NAME,  cache_dir=CACHE_DIR)
preprocessed_dataset = processor.preprocess(test_dataset)

In [ ]:
preprocessed_dataset[0].keys()

In [ ]:
prediction = summarizer.predict(preprocessed_dataset, num_gpus=0, batch_size=1, sentence_separator="\n")

In [ ]:
prediction

## Clean up temporary folders

In [ ]:
if os.path.exists(DATA_PATH):
    shutil.rmtree(DATA_PATH, ignore_errors=True)
if os.path.exists(CACHE_DIR):
    shutil.rmtree(CACHE_DIR, ignore_errors=True)
if USE_PREPROCSSED_DATA:
    if os.path.exists(PROCESSED_DATA_PATH):
        shutil.rmtree(PROCESSED_DATA_PATH, ignore_errors=True)